# Fine tuning pre-trained language models for text classification



## Overview
Fine-tuning pre-trained language models learnt with transformers has improved the state of the art in multiple NLP evaluation tasks(see [SuperGlue leader board](https://super.gluebenchmark.com/leaderboard)). Learning a language model is an unsupervised task where the model learns to predict the next word in a sequence given the previous words. Neural language models have been implemented as feed foorward networks, LSTMS (ELMo, ULMFit), and transformers-encoders (BERT) or decoders (Open AI GPT).

In this notebook we fine tune a BERT pre-trained language model to carry out a binary classification task where tweets are labelled as generated by bots or hurmans. 

<!-- The notebook is structured as follows:

- Motivation
- Setup
  - Libraries required
  - Dataset
- A glimpse on BERT tokenization 
- Fine tune the model
- Evaluate the BERT classifier 
-->

## Motivation
While word embeddings are learnt from large corpora, their use in neural models to solve specific tasks is limited to the input layer. So in practice a task-specific neural model is built almost from scratch because most of the model parameters are initialized randomly, and hence, these paremeters need to be optimized for the task at hand, requiring large sets of data to produce a high performance model.

Recent advances in neural language models (BERT or OPEN AI GPT) have shown evidence that task specific architectures are not longer necessary and transfering some internal representations (attention blocks) along with shallow feed forward networks is enough. 

In (Garcia et al.,2019) we presented an experimental study on the use of word embeddings as input of CNN architectures and Bi-LSTM to tackle the bot detection task and compare these results with fine-tuning pretrained language models. 

Evaluation results, presented in the figure below, show that fine-tuning language models yields overall better results than training specific neural architectures that are fed with mixture of: i) pre-trained contextualized word embeddings (ELMo), ii) pre-trained  context-indepedent word embeddings learnt from Common Crawl(FastText), Twitter (GloVe), and urban dictionary (word2vec), plus embeddings optimized by the neural network in the learning process. 

![Bot detection classification task](https://drive.google.com/uc?id=1rSzM544MK2QOezpvUKHfrxATbkEiyBHX)



**References**

Garcia-Silva, Andres, et al. "An Empirical Study on Pre-trained Embeddings and Language Models for Bot Detection." Proceedings of the 4th Workshop on Representation Learning for NLP (RepL4NLP-2019). 2019.

To cite this paper use the following BibTex entry: 

```
@inproceedings{garcia-silva-etal-2019-empirical,
    title = "An Empirical Study on Pre-trained Embeddings and Language Models for Bot Detection",
    author = "Garcia-Silva, Andres  and
      Berrio, Cristian  and
      G{\'o}mez-P{\'e}rez, Jos{\'e} Manuel",
    booktitle = "Proceedings of the 4th Workshop on Representation Learning for NLP (RepL4NLP-2019)",
    month = aug,
    year = "2019",
    address = "Florence, Italy",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/W19-4317",
    doi = "10.18653/v1/W19-4317",
    pages = "148--155",
}
```



# A glimpse on BERT


## Input representation

<img src="https://drive.google.com/uc?id=1tZS7sszhNtT3m25EZJkjC9PjRZDEPKGy" alt="Token embeddings, segment embeddings, and positional embeddings" width="500"/>

Image source: "Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). Bert: Pre-training of deep bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805."

## Pre-training learning objectives

### Masked language model

<img src="https://drive.google.com/uc?id=11GJiHlDeoKsShOwSJvMTq3fc8E8GxV4V" alt="Masked LM" width="500"/>

Image Source: https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270

### Next Sentence Prediction

<img src="https://drive.google.com/uc?id=1x0ckvgMwb5j3SfVNId-EyQgyDlt3C42h" alt="next sentence prediction" width="500"/>

Image source: http://jalammar.github.io/illustrated-bert/


## Contextualized word embeddings

<img src="https://drive.google.com/uc?id=1LozTCltkxXbkrE2M72r0lDUWEDXLEWZU" alt="Contextualized embeddings" width="600"/>

Image source: http://jalammar.github.io/illustrated-bert/


## Fine-tuning

fine-tuning BERT for a task just requires to incorporate one  additional  output  layer,  so a minimal number of parameters need to be learned from scratch.

In the figure below E represents the input embedding,Ti represents the contextual representation of token i, [CLS] is the special symbol for classification output, and [SEP] is the specialsymbol to separate non-consecutive token sequences

<img src="https://drive.google.com/uc?id=1wZPwbMNtHwf8g-7phWxwtJCxnTfPj-Ux" width="600"/>

Image source: Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). Bert: Pre-training of deep bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805.

To **fine-tune BERT for a sequence classification task** the transfomer output for the CLS token is used as the sequence representation. The transfomer output for the CLS token is connected to a one layer feed forward network that predicts the classification labels. All the BERT parameters and the FF network are fine-tune jointly to maximize the log-probability of the correct label.






# Experimental setup


## Transformers library

We use transformer from huggingface: https://github.com/huggingface/transformers

"Transformers provides state-of-the-art general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet, CTRL...) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between TensorFlow 2.0 and PyTorch."

In [0]:
!pip install transformers

     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 860kB 44.6MB/s 
     |████████████████████████████████| 1.0MB 33.3MB/s 
     |████████████████████████████████| 645kB 41.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=c1a8255c933b6acd9990e2eb3874ada7daf1996813060c31ad6e1d6f04a91ea0
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


## Dataset 

We use the bot detection dataset generated in (Garcia et al.,2019) that was built starting from an existing list of twitter accounts that were manually labelled as bots and humans. Then we use the twitter API to extract tweets from these account.  In total the dataset contains around 600K tweet, approximately half of them generated by bots, and the other half by humans. 

In this notebook we provide a complete version of the dataset (large) and a reduced one (small) to be able to run the notebook whithin the time frame, since **fine tuning BERT on the large version takes more than 5h**. 

- Large: 500k train and 100k test labeled tweets which is in the path: "'/content/gdrive/My Drive/09_BERT/Large_Dataset/"
- Small: 1k train and 100 test labeled tweets which is in the path: "'/content/gdrive/My Drive/09_BERT/Small_Dataset/"

### Mounting Google Drive

Let's access the google drive where we have stored the datasets and the pretrained models

When running the following cell a link will appear, following that link you will find an authorization code to be pasted below.
This is necessary to access the Google Drive account.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Set the dataset version
The enviroment variable DATA_DIR holds the path to the dataset 

In [25]:
%env DATA_DIR=/content/gdrive/My Drive/09_BERT/Small_Dataset/

#Uncomment the following line to use the large version of the dataset
#%env DATA_DIR=/content/gdrive/My Drive/09_BERT/Large_Dataset/

env: DATA_DIR=/content/gdrive/My Drive/09_BERT/Small_Dataset/
env: DATA_DIR=/content/gdrive/My Drive/09_BERT/Large_Dataset/


### Inspect the dataset

The dataset is in the tsv format expected by transfomer library. 

In [15]:
import os
import pandas as pd

test = pd.read_csv(os.environ["DATA_DIR"] + "dev.tsv", header=None, sep = '\t')
data = pd.DataFrame(test)
data.columns = ["index", "label", "mark", "tweet"]
data

,index,label,mark,tweet
0,0,1,a,Now Playing: ♬ Dick Curless - Evil Hearted Me ...
1,1,0,a,Not only are you comfortably swaddled in secur...
2,2,1,a,Follow @iAmMySign !!! Follow @iAmMySign our o...
3,3,0,a,These strawberry sandwich cookies are so easy ...
4,4,0,a,Do These Two Lines Match Up On Your Hands Here...
...,...,...,...,...
99995,99995,0,a,I Love The World Cup !!
99996,99996,0,a,CONFIRMED: Today is it for the season for one ...
99997,99997,0,a,Stack that paper up &amp; then make boss moves...
99998,99998,1,a,Wednesday Free Pick is Live! #freepicks #freet...


# Hands-on

## Tokenization 

Recent neural languages models use subword representations. ELMO relies on characters, Open AI GPT on byte pair encoding, and BERT on the word pieces algorithms. These **subword representations are combined when unseen words during training needs to be processed, hence avoiding the OOV problem**. 

BERT uses a 30k WordPieces vocabulary. 

Let us see how the BERT Tokenizer works

In [0]:
from transformers import *

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = input("Enter a word or a sentence: ")
print(tokenizer.tokenize(text))
print(tokenizer.encode(text))


Enter a word or a sentence: ecent neural languages models use subword representations. ELMO relies on
['ec', '##ent', 'neural', 'languages', 'models', 'use', 'sub', '##word', 'representations', '.', 'elm', '##o', 'relies', 'on']
[14925, 4765, 15756, 4155, 4275, 2224, 4942, 18351, 15066, 1012, 17709, 2080, 16803, 2006]


## Fine-Tuning the model

The next step would be to fine-tune the model.

Running the following script you can fine-tune the model and perform evaluation. While doing the evaluation the classification of the tweets on the test set is saved in the predictions.txt file that we will use later.

The most relevant parameters of the script are:
  - model type: the model that we are going to use, in this case BERT
  - model name or path: the name of the model or path storing a specific model.
  - task name: the task that we want to perform, in this case CoLA because we want to do classification.
  - ouput dir: the directory in which it stores the fine-tuned model.
  
You can try to change the parameters and see how it affects performance. 

This process is slow even though we reduced the dataset. You should expect that it takes around 1 minute.

In [0]:
!python /content/gdrive/My\ Drive/09_BERT/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name CoLA \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir "$DATA_DIR" \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1.0 \
    --save_steps 62500 \
    --overwrite_output_dir \
    --output_dir  ./Bert_Classifier/

11/14/2019 09:04:24 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/14/2019 09:04:25 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
11/14/2019 09:04:25 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "cola",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "p

If you trained with the small dataset you should see a final result such as mcc = 0.24.

On the other hand if you trained with the large one mcc increases to 0.70

The MCC score measures how well does the algorithm perform on both positive and negative predictions.

It gives more information than the accuracy or the f1 score.

This numbers ranges from -1 to 1 being 0 the random case, -1 the worst value and +1 the best value.

## Further Evaluation

Let's compute the metrics of our fine-tuned model to see how well it performs on the test set

In [0]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

preds = np.loadtxt("./Bert_Classifier/predictions.txt")
test = pd.read_csv(os.environ["DATA_DIR"] + "dev.tsv", header=None, sep = '\t')

print(classification_report(np.asarray(test[1]), preds))
print("Accuracy: ", accuracy_score(np.asarray(test[1]), preds))
print("MCC: ", matthews_corrcoef(test[1], preds))

              precision    recall  f1-score   support

           0       0.66      0.61      0.63        54
           1       0.58      0.63      0.60        46

    accuracy                           0.62       100
   macro avg       0.62      0.62      0.62       100
weighted avg       0.62      0.62      0.62       100

Accuracy:  0.62


You should see an accuracy of 62% and an f1-score of 62% which is a good result considering the size of the dataset.

The full model fine-tuned on the 500k tweets achieve the following metrics:

    - Accuracy = 0.85
    - Recall = 0.85
    - Precision = 0.86
    - Recall = 0.85



## Perform inference

Now let's take some random examples from our test set:

In [0]:
os.mkdir("./Test_Dataset/") # We are going to store the test dataset in this folder

test_evaluate = test[:4]
print(test_evaluate)
test_evaluate.to_csv("./Test_Dataset/dev.tsv", sep='\t', index=False, header=False)

   0  1  2                                                  3
0  0  1  a  Now Playing: ♬ Dick Curless - Evil Hearted Me ...
1  1  0  a  Not only are you comfortably swaddled in secur...
2  2  1  a  Follow @iAmMySign !!!  Follow @iAmMySign our o...
3  3  0  a  These strawberry sandwich cookies are so easy ...


If you want to perform inference with the larger model we provide an already trained version. You only have to change the argument in model_name_or path from Bert_Classifier_small to Bert_Classifier_Large

In [16]:
%env MODEL_PATH=./Bert_Classifier/

#Uncomment the following line to use the version of the model trained with the large dataset
#%env MODEL_PATH=/content/gdrive/My Drive/09_BERT/Bert_Classifier_Large/

env: MODEL_PATH=./Bert_Classifier/
env: MODEL_PATH=/content/gdrive/My Drive/09_BERT/Bert_Classifier_Large/


In [19]:
!python /content/gdrive/My\ Drive/09_BERT/run_glue.py \
    --model_type bert \
    --model_name_or_path "$MODEL_PATH" \
    --task_name CoLA \
    --do_eval \
    --do_lower_case \
    --data_dir ./Test_Dataset/ \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 1.0 \
    --save_steps 62500 \
    --output_dir  "$MODEL_PATH"

11/14/2019 11:51:29 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/14/2019 11:51:29 - INFO - transformers.configuration_utils -   loading configuration file /content/gdrive/My Drive/09_BERT/Bert_Classifier_Large/config.json
11/14/2019 11:51:29 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "cola",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

11/14/2019 11:51:29 - INFO - transformers.tokenization_utils -   Model name

mcc

Let's see if the model has correctly classified the examples:

In [0]:
import os

results = np.loadtxt(os.environ['MODEL_PATH'] + "predictions.txt")
for i,t in enumerate(test_evaluate[3]):
    print(t + " --> ", "BOT" if results[i]> 0.5 else "NOT A BOT")

Now Playing: ♬ Dick Curless - Evil Hearted Me ♬ https://t.co/fzgP9IRt2h -->  BOT
Not only are you comfortably swaddled in security today, it’s ... More for Capricorn https://t.co/MVCHEli4g1 -->  NOT A BOT
Follow @iAmMySign !!!  Follow @iAmMySign our official page for the whole Zodiac.  Follow @iAmMySign !!!  Follow @iAmMySign !!! -->  BOT
These strawberry sandwich cookies are so easy to make and so tasty! Perfect for #MothersDay https://t.co/Uq7cooR2y7 via @iamthemaven -->  NOT A BOT
